# Classification pipeline for a single glacier

Full glacier-snow-cover-mapping classification pipeline for Sentinel-2 TOA, Sentinel-2 SR, and Landsat 8/9 images. 

## Requirements:
1. Google Earth Engine (GEE) account: used to query imagery and the DEM (if no DEM is provided). Sign up for a free account [here](https://earthengine.google.com/new_signup/). 

2. Google Drive folder: Create a folder where output snow cover statistics will be saved. Enter the name of this folder as the `out_folder` variable below. If you don't create the folder ahead of time, duplicates of the same folder will be created for each output file!

## Notes on GEE job submissions

GEE enforces [user quotas](https://developers.google.com/earth-engine/guides/usage) on memory usage (default = 10 MB per query) and the number of concurrent requests (default = 40). The workflow below mitigates the risk of exceeding these limits by splitting tasks into smaller date ranges.While GEE handles queued tasks efficiently, it's our job to manage the size of the individual jobs (i.e., image collections). To reduce task size, the date range is divided before running the workflow and submitting export tasks based on the size of the defined area of interest (AOI):

- AOI < 500 km$^2$: split by month
- 700 km$^2$ <= AOI < 1100 km$^2$: split by week
- AOI >= 1100 km$^2$: split by day

Especially for the largest glaciers, this means that some exports may be empty, depending on image availability. To compile all results and remove empty CSVs from your Google Drive folder, see the `post_processing.ipynb` notebook in this repo. 

## Define image search settings and paths

In [1]:
import os
import ee
import geemap
import sys
import numpy as np

# -----Define Google Drive folder for outputs
# Note: Make sure this folder already exists and is the only folder in your "My Drive" with that name. 
out_folder = 'glacier_snow_cover_exports'

# -----Import pipeline utilities
# Assumes pipeline_utils.py is in the same folder as this notebook
script_path = os.getcwd()
sys.path.append(script_path)
import pipeline_utils as utils

# -----Define image search settings
# Date and month ranges (inclusive)
date_start = '2013-06-01'
date_end = '2024-10-31'
month_start = 6
month_end = 10
# Minimum fill portion of the AOI (0–100), used to remove images after mosaicking by day. 
min_aoi_coverage = 70
# Whether to mask clouds using the respective cloud mask via the geedim package
mask_clouds = True

/Users/jukesliu/micromamba/envs/gscm_gee/lib/python3.11/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Authenticate and/or Initialize Google Earth Engine (GEE)

Replace the project ID with your GEE project. Default = `ee-{GEE-username}`

In [2]:
project_id = "snow-cover-mapping-463217"
# project_id = "ee-raineyaberle"

try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()
    ee.Initialize(project=project_id)

## Select the Area of Interest (AOI) from the GLIMS dataset

This cell will plot the GLIMS dataset on a map. To find a glacier, click on the wrench in the upper right toolbox of the map, and use the "Inspector" to click on a polygon and view the its properties. Right click on the "glac_id" property to highlight and then copy. Replace the `glac_id` variable below with your selected site. 

In [3]:
# Load the GLIMS dataset, add to the map
glims = ee.FeatureCollection('GLIMS/20230607')

# Select your study site from the GLIMS dataset
# glac_id = 'G211100E60420N'
# aoi = glims.filter(ee.Filter.eq('glac_id', glac_id))

# => Test with the largest AK glacier (Malaspina)
glims_ak = (glims.filter(ee.Filter.eq('gtng_o1reg', 1))) # filter to AK
glims_ak_largest = glims_ak.filter(ee.Filter.eq('area', glims_ak.aggregate_max('area'))) # Get the largest AK glacier
aoi = glims_ak_largest
# use GLIMS ID for output file names
if type(aoi)==ee.featurecollection.FeatureCollection:
    glac_id = ee.FeatureCollection(aoi).aggregate_array('glac_id').get(0).getInfo()
else:
    glac_id = aoi.get('glac_id').getInfo() 
print("Glacier ID used for output file names:", glac_id)
                                 
# Grab the geometry
# aoi = glims.filter(ee.Filter.eq('glac_id', glac_id))
aoi = aoi.geometry()
aoi_area = aoi.area().getInfo() # save area [m^2] for splitting date ranges later
print(f"Glacier area = {int(aoi_area/1e6)} km2")

# Create a Map
Map = geemap.Map()
Map.addLayer(glims, {'color': 'blue', 'opacity':  0.5}, 'GLIMS/20230607')
Map.addLayer(aoi, {'color': 'orange', 'opacity': 0.8}, 'AOI')
Map.centerObject(aoi)
Map

Glacier ID used for output file names: G219787E60289N
Glacier area = 3605 km2


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Load the Digital Elevation Model (DEM)

Default: use the ArcticDEM Mosaic where there is > 90 % coverage. Otherwise, use the NASADEM. For sites that use the ArcticDEM Mosaic, elevations are reprojected to the EGM96 geoid to match the vertical datum of NASADEM. 

In [4]:
# Query GEE for DEM
dem = utils.query_gee_for_dem(aoi)

# Add DEM to map
# grab min and max elevations for color limits
minMax = dem.reduceRegion(reducer=ee.Reducer.minMax(),
                          geometry=aoi, 
                          scale=30,
                          maxPixels=1e9,
                          bestEffort=True)
elev_min = minMax.get('elevation_min')
elev_max = minMax.get('elevation_max')
print(f'Elevation range = {int(elev_min.getInfo())} to {int(elev_max.getInfo())} m')
# colors based on the "terrain" palette from matplotlib
palette = ['#333399', '#0d7fe5', '#00be90','#55dd77','#c6f48e','#e3db8a','#aa926b','#8e6e67','#c6b6b3','#ffffff']
Map.addLayer(dem, {'palette': palette, 'min': elev_min, 'max': elev_max}, 'DEM')


Querying GEE for DEM
ArcticDEM coverage = 96 %
Using ArcticDEM Mosaic
Elevation range = -1 to 5952 m


## Run the classification pipeline for each dataset

### Sentinel-2 Top of Atmosphere (TOA): 2016 onwards

In [5]:
import importlib
importlib.reload(utils)

<module 'pipeline_utils' from '/Users/jukesliu/Documents/GitHub/gscm_gee/pipeline_utils.py'>

In [6]:
dataset = "Sentinel-2_TOA"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, glac_id)

AOI >= 1100 km2 — splitting date range by day.
Number of date ranges = 1376

 ('2016-06-01', '2016-06-02')
Querying GEE for Sentinel-2_TOA image collection
Classifying image collection
Calculating snow cover statistics
Exporting snow cover statistics to glacier_snow_cover_exports Google Drive folder with file name: G219787E60289N_Sentinel-2_TOA_snow_cover_stats_2016-06-01_2016-06-02
To monitor tasks, see your Google Cloud Console or GEE Task Manager: https://code.earthengine.google.com/tasks

 ('2016-06-02', '2016-06-03')
Querying GEE for Sentinel-2_TOA image collection
Classifying image collection
Calculating snow cover statistics
Exporting snow cover statistics to glacier_snow_cover_exports Google Drive folder with file name: G219787E60289N_Sentinel-2_TOA_snow_cover_stats_2016-06-02_2016-06-03
To monitor tasks, see your Google Cloud Console or GEE Task Manager: https://code.earthengine.google.com/tasks

 ('2016-06-03', '2016-06-04')
Querying GEE for Sentinel-2_TOA image collection
Cl

### Sentinel-2 Surface Reflectance (SR): 2019 onwards

In [ ]:
dataset = "Sentinel-2_SR"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, glac_id)

### Landsat 8/9 SR: 2013 onwards

In [ ]:
dataset = "Landsat"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, glac_id)

_Fin!_

# Run the classification on a selection of glaciers

Select an entire region of glaciers by RGI region: https://rgitools.readthedocs.io/en/latest/dems_subregions.html

In [3]:
# Load the GLIMS dataset, add to the map
glims = ee.FeatureCollection('GLIMS/20230607')

# Filter by region and subregion
glims_r1 = (glims.filter(ee.Filter.eq('gtng_o1reg', 1))) # filter by RGI region (e.g., 1 = Alaska)
glims_r2 = (glims_r1.filter(ee.Filter.eq('gtng_o2reg', 5))) # filter RGI subregion (e.g., 5 = St. Elias)
id_list = glims_r2.aggregate_array('glac_id') # grab list of GLIMS IDs
id_list = id_list.getInfo()
print('Number of glacier sites: ',len(id_list))

22274


In [7]:
id_list = id_list[10:] # start at a certain id

In [ ]:
# for each site id
for i in range(0,len(id_list)):
    glac_id = id_list[i]
    print("Glacier ID used for output file names:", glac_id)

    # grab glacier area of interest
    aoi = glims.filter(ee.Filter.eq('glac_id', glac_id))
    aoi = aoi.geometry()
    aoi_area = aoi.area().getInfo() # save area [m^2] for splitting date ranges later
    print(f"Glacier area = {aoi_area/1e6} km2")

    dem = utils.query_gee_for_dem(aoi) # query DEM

    # run each dataset
    for dataset in ['Sentinel-2_TOA','Sentinel-2_SR','Landsat']:
        utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, glac_id)

Glacier ID used for output file names: G219885E61633N
Glacier area = 0.1906191048495416 km2

Querying GEE for DEM
ArcticDEM coverage = 99 %
Using ArcticDEM Mosaic
AOI area < 500 km2 — splitting date range by month.
Number of date ranges = 101

 ('2016-06-01', '2016-06-30')
Querying GEE for Sentinel-2_TOA image collection
Classifying image collection
Calculating snow cover statistics
Exporting snow cover statistics to glacier_snow_cover_exports Google Drive folder with file name: G219885E61633N_Sentinel-2_TOA_snow_cover_stats_2016-06-01_2016-06-30
To monitor tasks, see your Google Cloud Console or GEE Task Manager: https://code.earthengine.google.com/tasks

 ('2016-07-01', '2016-07-31')
Querying GEE for Sentinel-2_TOA image collection
Classifying image collection
Calculating snow cover statistics
Exporting snow cover statistics to glacier_snow_cover_exports Google Drive folder with file name: G219885E61633N_Sentinel-2_TOA_snow_cover_stats_2016-07-01_2016-07-31
To monitor tasks, see your

## Check and cancel batch tasks:

In [92]:
ee.batch.Task.list()

[<Task UN575D5UBQ7LOMRH2EZ7HCL3 EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_cover_stats_2020-04-01_2020-04-30 (COMPLETED)>,
 <Task HEFNVLAN4VOHTK3MMDCJT3BR EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_cover_stats_2020-03-01_2020-03-31 (COMPLETED)>,
 <Task YUBYJSIAVS2VHNCQUKH445P7 EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_cover_stats_2020-02-01_2020-02-29 (COMPLETED)>,
 <Task MLEPTKLRATPGDAU3UCWN6QAB EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_cover_stats_2020-01-01_2020-01-31 (COMPLETED)>,
 <Task G7RXFI23FQJP3UZHIERPITOF EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_cover_stats_2019-12-01_2019-12-31 (COMPLETED)>,
 <Task POZGSK2MOSXRCVPKOX2FPQJI EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_cover_stats_2019-11-01_2019-11-30 (COMPLETED)>,
 <Task DQMP5RJ3XI4CWZ73NPTKIRXP EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_cover_stats_2019-10-01_2019-10-31 (COMPLETED)>,
 <Task XUID4YFLAA6UHCEXRRQZ7JOH EXPORT_FEATURES: G219885E61633N_Sentinel-2_SR_snow_

In [93]:
print(len(ee.batch.Task.list()))

8552


In [5]:
id_list.index('G219885E61633N')

10